In [1]:
import sys

from sklearn.model_selection import train_test_split

sys.path.append('../')
import pandas as pd
from tqdm import tqdm
from utils.basic_utils import dump_pickle, load_pickle
from utils.display_utils import info
import gc
import numpy as np

In [156]:
# mark = "speech,music,vehicle" max_per_class=6200, 1200
# mark ='cat,cough,laughter'    max_per_class=75, 25
# mark ='cat,cough,laughter,aircraft' max_per_class=75, 25
mark ='esc10(as)'
split='test'


In [157]:
# 29_classes = 33 classes + Silence + Speech - 'Glass' - 'Meow' - 'Pig' - 'Wild animals' - 'Silence' - 'Tap' - 'Scratch'
classes28 =  ['Sneeze', 'Dog', 'Clock',  'Baby cry, infant cry', 'Chicken, rooster', 'Rain', 'Waves, surf', 'Firecracker', 'Helicopter', 'Chainsaw'] 
src_classes = ['Speech', 'Music', 'Baby cry, infant cry', 'Cough', 'Sneeze', 'Walk, footsteps', 'Clapping', 'Cheering', 'Dog', 'Cat', 'Cowbell',  'Chicken, rooster', 'Frog', 'Wind', 'Rain', 'Raindrop', 'Rain on surface', 'Ocean', 'Waves, surf', 'Vehicle', 'Helicopter', 'Engine', 'Chainsaw', 'Water tap, faucet', 'Keys jangling', 'Clock', 'Firecracker', 'Squeal']
as_label_idx = pd.read_csv('/home/alhasan/workdir/SED/data/AudioSet/class_labels_indices.csv')
as_label_idx = as_label_idx.loc[as_label_idx['display_name'].isin(classes28)]
classes28_idx = as_label_idx['index'].to_list()


labels_dict = load_pickle('/home/alhasan/workdir/SED/data/AudioSet/PANN_labels_all(new).pkl')['labels']
embeddings_dict = load_pickle('/home/alhasan/workdir/SED/data/AudioSet/openl3_embeddings_500_512.pkl')['embeddings']
files = [f for f in embeddings_dict.keys() if f in labels_dict.keys()]

In [158]:
as_label_idx

,index,mid,display_name
23,23,/t/dd00002,"Baby cry, infant cry"
49,49,/m/01hsr_,Sneeze
74,74,/m/0bt9lr,Dog
99,99,/m/09b5t,"Chicken, rooster"
289,289,/m/06mb1,Rain
295,295,/m/034srq,"Waves, surf"
339,339,/m/09ct_,Helicopter
347,347,/m/01j4z9,Chainsaw
406,406,/m/01x3z,Clock
433,433,/g/122z_qxw,Firecracker


In [159]:
# src_classes=classes28

In [160]:
classes28_idx

[23, 49, 74, 99, 289, 295, 339, 347, 406, 433]

In [161]:
train_files=[]
test_files=[]

In [162]:
for c in src_classes:
    # get all files of c
    s_files = [f for f in files if f.split("/")[0] == c]
    # split them into train_test
    cnt_c = len(s_files)
    print(f"{cnt_c:5d} samples in class {c} ")

    0 samples in class Speech 
 5379 samples in class Music 
   42 samples in class Baby cry, infant cry 
   40 samples in class Cough 
   37 samples in class Sneeze 
   50 samples in class Walk, footsteps 
   48 samples in class Clapping 
   57 samples in class Cheering 
  255 samples in class Dog 
  354 samples in class Cat 
   50 samples in class Cowbell 
   52 samples in class Chicken, rooster 
   51 samples in class Frog 
  460 samples in class Wind 
  294 samples in class Rain 
   54 samples in class Raindrop 
   48 samples in class Rain on surface 
   55 samples in class Ocean 
   58 samples in class Waves, surf 
 1091 samples in class Vehicle 
   48 samples in class Helicopter 
  463 samples in class Engine 
   54 samples in class Chainsaw 
   61 samples in class Water tap, faucet 
   53 samples in class Keys jangling 
   93 samples in class Clock 
   37 samples in class Firecracker 
  153 samples in class Squeal 


In [163]:
for c in src_classes:
    # get all files of c
    s_files = [f for f in files if f.split("/")[0] == c]
    # split them into train_test
    cnt_c = len(s_files)
    test_cnt = cnt_c // 4
    # append train to train list and test to test list
    print(f"keeping {cnt_c} files from class {c}")
    train_files += s_files[:-test_cnt]
    test_files += s_files[-test_cnt:]
print(f"# train files = {len(train_files)}")
print(f"# test files = {len(test_files)}")

keeping 0 files from class Speech
keeping 5379 files from class Music
keeping 42 files from class Baby cry, infant cry
keeping 40 files from class Cough
keeping 37 files from class Sneeze
keeping 50 files from class Walk, footsteps
keeping 48 files from class Clapping
keeping 57 files from class Cheering
keeping 255 files from class Dog
keeping 354 files from class Cat
keeping 50 files from class Cowbell
keeping 52 files from class Chicken, rooster
keeping 51 files from class Frog
keeping 460 files from class Wind
keeping 294 files from class Rain
keeping 54 files from class Raindrop
keeping 48 files from class Rain on surface
keeping 55 files from class Ocean
keeping 58 files from class Waves, surf
keeping 1091 files from class Vehicle
keeping 48 files from class Helicopter
keeping 463 files from class Engine
keeping 54 files from class Chainsaw
keeping 61 files from class Water tap, faucet
keeping 53 files from class Keys jangling
keeping 93 files from class Clock
keeping 37 files fr

In [164]:
embeddings_train =[]
as_labels_train = []

In [165]:
if split =='train':
    f_lst = train_files
    max_per_class=60  

else:
    f_lst = test_files
    max_per_class=15



In [166]:
for f in tqdm(f_lst):
#     n_segments = 175565
    n_segments = min(len(embeddings_dict[f]), len(labels_dict[f]))
    embeddings_train.append(embeddings_dict[f][:n_segments])
    as_labels_train.append(labels_dict[f][:n_segments])

# concatenate
as_labels_train = np.concatenate(as_labels_train, axis=0)
embeddings_train = np.concatenate(embeddings_train, axis=0)

info(f"len files labels: {len(as_labels_train)}")
info(f"embeddings: {len(embeddings_train)}")
info(f"len unique files labels: {len(set(as_labels_train))}")


100%|██████████| 2349/2349 [00:00<00:00, 142960.66it/s]

[2022-04-14 19:10:14.430203] -- len files labels: 44281
[2022-04-14 19:10:14.430418] -- embeddings: 44281
[2022-04-14 19:10:14.437849] -- len unique files labels: 266


In [167]:
len([1 for x in as_labels_train if x == 474])

0

In [168]:
 # train: 175565
read_per_class={c:0 for c in set(classes28_idx)}

classes_28_samples = []
info ("Collecting G28...") 
for i in tqdm(range(len(as_labels_train))):
    c=as_labels_train[i]
    if c in classes28_idx and read_per_class[c]<max_per_class:
        classes_28_samples.append(i)
        read_per_class[c]+=1
info(f"samples to use: {len(classes_28_samples)}")
labels28_train = [as_labels_train[i] for i in classes_28_samples]
embeddings28_train = [embeddings_train[i] for i in classes_28_samples]

assert(len(set(labels28_train)) == len(classes28)), f"Expected {len(classes28)} classes got {len(set(labels28_train))}"

[2022-04-14 19:10:14.608213] -- Collecting G28...


100%|██████████| 44281/44281 [00:00<00:00, 427092.43it/s]

[2022-04-14 19:10:14.718477] -- samples to use: 134


In [169]:
[x for x in set(classes28_idx) if x not in labels28_train]

[]

In [170]:
info(f"Numbre of labels in G28: {len(set(labels28_train))}")
info(f"Numbre of samples in G28: {len(classes_28_samples)}")

[2022-04-14 19:10:14.983044] -- Numbre of labels in G28: 10
[2022-04-14 19:10:14.983339] -- Numbre of samples in G28: 134


In [171]:
panns_dataset28_train = {'classes_28': classes28,
                         'classes28_audioset_index': classes28_idx,
                         'labels': labels28_train,
                         'embeddings': embeddings28_train
                        }

In [172]:
info("Saving G28 train to disk")
ds28_path = f'/home/alhasan/workdir/SED/data/AudioSet/openl3_embeddings_500_512_panns(classes_{mark}_{split}).pkl'
dump_pickle(panns_dataset28_train, ds28_path)
info("done")

[2022-04-14 19:10:15.200531] -- Saving G28 train to disk
[2022-04-14 19:10:15.204334] -- done


In [13]:
data = load_pickle('/home/alhasan/workdir/SED/data/AudioSet/openl3_embeddings_500_512_panns(classes_esc10(as)_train).pkl')

In [16]:
dd=data['labels']
len([x for x in dd if x==74])

60

In [14]:
lst = data['embeddings'].keys()
lst = list(set([l.split('/')[0] for l in lst]))

AttributeError: 'list' object has no attribute 'keys'

In [10]:
lst

['Scratch',
 'Firecracker',
 'Rain on surface',
 'Clock',
 'Cheering',
 'Raindrop',
 'Pig',
 'Ocean',
 'Chicken, rooster',
 'Baby cry, infant cry',
 'Sneeze',
 'Squeal',
 'Wind',
 'Cat',
 'Glass',
 'Rain',
 'Frog',
 'Engine',
 'Meow',
 'Chainsaw',
 'Dog',
 'Cough',
 'Walk, footsteps',
 'Waves, surf',
 'Wild animals',
 'Tap',
 'Music',
 'Clapping',
 'Water tap, faucet',
 'Cowbell',
 'Helicopter',
 'Vehicle',
 'Keys jangling']